## 2) 바다나우 어텐션(Bahdanau Attention)

지난 챕터에서 어텐션 메커니즘의 목적과 어텐션 메커니즘의 일종인 닷 프로덕트 어텐션(루옹 어텐션)의 전체적인 개요를 살펴보고, 마지막에 표를 통해 그 외에도 다양한 어텐션 메커니즘이 존재한다고 소개하였다. 이번 챕터에서는 닷 프로덕트 어텐션보다는 조금 더 복잡하게 설계된 바다나우 어텐션 메커니즘을 이해해보자.



### 1.바다나우 어텐션 함수(Bahdanau Attention Function)

어텐션 메커니즘을 함수 Attention()으로 정의하였을 때, 바다나우 어텐션 함수의 입, 출력은 다음과 같이 정의할 수 있다.

**Attention(Q, K, V) = Attention Value**

* t = 어텐션 메커니즘이 수행되는 디코더 셀의 현재 시점을 의미.
* Q = Query : t-1 시점의 디코더 셀에서의 은닉 상태
* K = Keys : 모든 시점의 인코더 셀의 은닉 상태들
* V = Values : 모든 시점의 인코더 셀의 은닉 상태들 

여기서는 어텐션 함수의 Query가 디코더 셀의 t 시점의 은닉 상태가 아니라 t-1 시점의 은닉 상태임을 주목하자.



### 2.바다나우 어텐션(Bahdanau Attention)

바다나우 어텐션의 연산 순서를 이해해보자.



#### 1) 어텐션 스코어(Attention Score)를 구한다.

<img src = 'https://wikidocs.net/images/page/73161/%EB%B0%94%EB%8B%A4%EB%82%98%EC%9A%B0%EC%96%B4%ED%85%90%EC%85%981.PNG' width = 60%>

인코더의 시점(time step)을 각각 1, 2, ..., N이라고 하였을 때 인코더의 은닉 상태(hidden state)를 각각 $h_{1}, h_{2}, ..., h_{N}$이라고 하자. 디코더의 현재 시점(time step) t에서의 디코더의 은닉 상태(hidden state)를 $s_{t}$라고 하자. 또한 여기서는 인코더의 은닉 상태와 디코더의 은닉 상태의 차원이 같다고 가정한다. 위의 그림의 경우에는 인코더의 은닉 상태와 디코더의 은닉 상태가 동일하게 차원이 4이다.

앞서 루옹 어텐션에서는 Query로 디코더의 t 시점의 은닉 상태를 사용한 것과는 달리 이번에는 t-1 시점의 은닉 상태 $s_{t-1}$를 사용한다. 바다나우 어텐션의 어텐션 스코어 함수. 즉 $s_{t-1}$과 인코더의 i번째 은닉 상태의 어텐션 스코어 계산 방법은 아래와 같다.

$score(s_{t-1},\ h_{i}) = W_{a}^{T}\ tanh(W_{b}s_{t-1}+W_{c}h_{i})$

단, $W_{a}, W_{b}, W_{c}$는 학습 가능한 가중치 행렬이다. 어차피 $s_{t-1}$과 $h_{1}, h_{2}, h_{3}, h_{4}$의 어텐션 스코어를 각각 구해야하므로 병렬 연산을 위해 $h_{1}, h_{2}, h_{3}, h_{4}$를 하나의 행렬 $H$로 두겠다. 수식은 다음과 같이 변경된다.

$score(s_{t-1},\ H) = W_{a}^{T}\ tanh(W_{b}s_{t-1}+W_{c}H)$

이제 이 수식을 그림을 통해 이해해보자. 우선 $W_{b}s_{t-1}$와 $W_{c}H$를 각각 구하면 다음과 같다.

<img src = 'https://wikidocs.net/images/page/73161/%EB%B0%94%EB%8B%A4%EB%82%98%EC%9A%B0%EC%96%B4%ED%85%90%EC%85%982.PNG' width = 60%>

이들을 더한 후, 하이퍼볼릭탄젠트 함수를 지나도록 한다.

<img src = 'https://wikidocs.net/images/page/73161/%EB%B0%94%EB%8B%A4%EB%82%98%EC%9A%B0%EC%96%B4%ED%85%90%EC%85%983.PNG' width = 60%>

지금까지 진행된 연산의 수식은 다음과 같다.

$tanh(W_{b}s_{t-1}+W_{c}H)$

이제 $W_{a}^{T}$와 곱하여 $s_{t-1}$과 $h_{1}, h_{2}, h_{3}, h_{4}$의 유사도가 기록된 어텐션 스코어 벡터 $e^{t}$를 얻는다.

<img src = 'https://wikidocs.net/images/page/73161/%EB%B0%94%EB%8B%A4%EB%82%98%EC%9A%B0%EC%96%B4%ED%85%90%EC%85%984.PNG' width = 60%>

$e^{t} = W_{a}^{T}\ tanh(W_{b}s_{t-1}+W_{c}H)$




#### 2) 소프트맥스(softmax) 함수를 통해 어텐션 분포(Attention Distribution)를 구한다.

<img src = 'https://wikidocs.net/images/page/73161/%EC%96%B4%ED%85%90%EC%85%98%EB%94%94%EC%8A%A4%ED%8A%B8%EB%A6%AC%EB%B7%B0%EC%85%98.PNG' width = 60%>

$e^{t}$에 소프트맥스 함수를 적용하여, 모든 값을 합하면 1이 되는 확률 분포를 얻어낸다. 이를 어텐션 분포(Attention Distribution)라고 하며, 각각의 값은 어텐션 가중치(Attention Weight)라고 한다.



#### 3) 각 인코더의 어텐션 가중치와 은닉 상태를 가중합하여 어텐션 값(Attention Value)을 구한다.

<img src = 'https://wikidocs.net/images/page/73161/%EC%BB%A8%ED%85%8D%EC%8A%A4%ED%8A%B8%EB%B2%A1%ED%84%B0.PNG' width = 60%>

이제 지금까지 준비해온 정보들을 하나로 합치는 단계이다. 어텐션의 최종 결과값을 얻기 위해서 각 인코더의 은닉 상태와 어텐션 가중치값들을 곱하고, 최종적으로 모두 더한다. 요약하면 가중합(Weighted Sum)을 한다고 말할 수도 있겠다. 이 벡터는 인코더의 문맥을 포함하고 있다고 하여, **컨텍스트 벡터(context vector)**라고 부른다.



#### 4) 컨텍스트 벡터로부터 $s_{t}$를 구한다.

기존의 LSTM이 $s_{t}$를 구할 때를 아래 그림을 통해 상기해보자. 기존의 LSTM은 이전 시점의 셀로부터 전달받은 은닉 상태 $s_{t-1}$과 현재 시점의 입력 $x_{t}$를 가지고 연산하였다. 아래의 LSTM은 seq2seq의 디코더이며 현재 시점의 입력 $x_{t}$는 임베딩된 단어 벡터이다.

<img src = 'https://wikidocs.net/images/page/73161/LSTM.PNG' width = 60%>

그렇다면 어텐션 메커니즘에서는 어떨까? 아래의 그림은 바다나우 어텐션 메커니즘에서는 컨텍스트 벡터와 현재 시점의 입력된 단어의 임베딩 벡터를 연결(concatenate)하고, 현재 시점의 새로운 입력으로 사용하는 모습을 보여준다. 그리고 이전 시점의 셀로부터 전달받은 은닉 상태 $s_{t-1}$과 현재 시점의 새로운 입력으로부터 $s_{t}$를 구한다.

다시 말해 기존의 LSTM이 임베딩된 단어 벡터를 입력으로 하는 것에서 컨텍스트 벡터와 임베딩된 단어 벡터를 연결(concatenate)하여 입력으로 사용하는 것이 달라졌다.

<img src = 'https://wikidocs.net/images/page/73161/%EB%B0%94%EB%8B%A4%EB%82%98%EC%9A%B0%EC%96%B4%ED%85%90%EC%85%985.PNG' width = 60%>

이후에는 어텐션 메커니즘을 사용하지 않는 경우와 동일하다. $s_{t}$는 출력층으로 전달되어 현재 시점의 예측값을 구하게 된다. 같은 내용이지만, 다른 사람의 설명을 함께 보면 이해에 더 도움이 될 것 같아 아래의 영상을 추천한다.

허민석님의 바다나우 어텐션 강의 영상 : https://www.youtube.com/watch?v=WsQLdu2JMgI


참고 자료 :

https://hcnoh.github.io/2018-12-11-bahdanau-attention

https://medium.com/analytics-vidhya/neural-machine-translation-using-bahdanau-attention-mechanism-d496c9be30c3